# **East Africa drought study - analysis of model data**

_Model fitting is less stable when using lognormal - carry out transformation separately_

In [ ]:
source("../wwa_nonstationary_fitting.r")

prep_window <- function(rc = c(1,1), w = 4, h = 4) { options(repr.plot.width = rc[2]*w, repr.plot.height = rc[1]*h, repr.plot.res = 200); par(mfrow = rc, pch = 20) }

---
## **Prep & testing**

In [ ]:
# load data, fix values for factual & counterfactual climate

df <- read.csv("data/gmst-nino-cpc.csv")

gmst_2022 <- df[df$year == 2022, "gmst"]
gmst_cf <- gmst_2022 - 1.2

ondnino_2022 <- df[df$year == 2022, "nino_ond"]
ondnino_cf <- ondnino_2022

In [ ]:
# wrapper function to get bootstrapped confidence intervals for spreadsheet
bootstrap_obs_results <- function(mdl, cov1, cov2, cov1_cf = NA, cov2_cf = NA, seed = 101, nsamp = 1000, dp = 5, convert_logs = T) {

    # create internal function to give required results
    get_mdl_values <- function(mdl, ev, cov1, cov2, cov1_cf = NA, cov2_cf = NA) {
        
        pars <- mdl$par
        
        rp <- 1/map_to_u(mdl, x = event_value, cov1 = cov1, cov2 = cov2)
        rp_cf <- 1/map_to_u(mdl, x = event_value, cov1 = cov1_cf, cov2 = cov2_cf)
        rl_cf <- map_from_u(1/rp, mdl, cov1 = cov1_cf, cov2 = cov2_cf)
        
        # if using log values, convert back to natural units
        if(convert_logs) {
            if(substr(mdl$varnm,1,5) == "log10") {
                ev <- 10^ev
                rl_cf <- 10^rl_cf
            } else if(substr(mdl$varnm,1,3) == "log") {
                ev <- exp(ev)
                rl_cf <- exp(rl_cf)
            }
        }
        c(pars, 
          "dispersion" = unname(pars["sigma0"] / pars["mu0"]),
          "event_magnitude" = ev,
          "return_period" = rp,
          "probability_ratio" = rp_cf/rp,
          "abs_change_in_intensity" = (ev - rl_cf),
          "rel_change_in_intensity" = ((ev - rl_cf) / rl_cf) * 100
         )   
    }
    
    # get best estimate from the observed data
    event_value <- mdl$x[mdl$ev_idx]
    mdl_res <- get_mdl_values(mdl, event_value, cov1, cov2, cov1_cf, cov2_cf)
    mdl_df <- setNames(data.frame(mdl$x, mdl$cov1, mdl$cov2), c(mdl$varnm, mdl$covnm_1, mdl$covnm_2)) 
    
    # get bootstrap sample
    set.seed(seed)    
    boot_res <- sapply(1:nsamp, function(i) {
        boot_df <- mdl_df[sample(1:nrow(mdl_df), nrow(mdl_df), replace = T),]
        boot_mdl <- fit_ns(mdl$dist, mdl$type, boot_df, varnm = mdl$varnm, covnm_1 = mdl$covnm_1, covnm_2 = mdl$covnm_2, lower = mdl$lower)
        get_mdl_values(boot_mdl, event_value, cov1, cov2, cov1_cf, cov2_cf)
    })
    boot_qq <- t(rbind("bestimate" = mdl_res, apply(boot_res, 1, quantile, c(0.025, 0.975), na.rm = T)))
    if(!is.na(dp)) boot_qq <- round(boot_qq, dp)
    return(boot_qq)
}

#### **Check results against Climate Explorer**

In [ ]:
# fit test model to log precip - should give same results as Climate Explorer
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "log10pr_ond", covnm_1 = "gmst", lower = T),
                        cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = F)

In [ ]:
# test same model using lognormal fit - should give comparable disp/RP/PR
bootstrap_obs_results(fit_ns("lnorm", "fixeddisp", df, varnm = "pr_ond", covnm_1 = "gmst", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = F)

#### **Instability in lognormal fit**

_Some instability in the model fit when using lognormal distribution: prefer to use log_pr & convert_

In [ ]:
bootstrap_obs_results(fit_ns("lnorm", "fixeddisp", df, varnm = "pr_ond", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

In [ ]:
# pr_ond ~ gmst + nino_ond
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "log10pr_ond", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

In [ ]:
# confirm: if we use natural log, we get almost same fitted parameters as using lognormal, but without the model instability
df["lognpr_ond"] <- log(df$pr_ond)
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "lognpr_ond", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

---
## **Observations**

In [1]:
source("../wwa_nonstationary_fitting.r")

# load covariate data
df <- merge(read.csv("ts/EA-drought_gmst-smoothed.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "gmst")),
            read.csv("ts/EA-drought_detrended-nino-ond.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "nino_ond")))

# fix covariate values
gmst_2022 <- df[df$year == 2022, "gmst"]
gmst_cf <- gmst_2022 - 1.2

ondnino_2022 <- ondnino_cf <- df[df$year == 2022, "nino_ond"]

### **Results for spreadsheet**

In [2]:
# wrapper function to get bootstrapped confidence intervals for spreadsheet
bootstrap_obs_results <- function(mdl, cov1, cov1_cf, cov2 = NA, cov2_cf = NA, seed = 101, nsamp = 1000, dp = 5, convert_logs = T, spreadsheet_format = T) {

    # create internal function to give required results
    get_mdl_values <- function(mdl, ev, cov1, cov2, cov1_cf = NA, cov2_cf = NA) {
        
        pars <- mdl$par
        
        rp <- 1/map_to_u(mdl, x = event_value, cov1 = cov1, cov2 = cov2)
        rp_cf <- 1/map_to_u(mdl, x = event_value, cov1 = cov1_cf, cov2 = cov2_cf)
        rl_cf <- map_from_u(1/rp, mdl, cov1 = cov1_cf, cov2 = cov2_cf)
        
        # if using log values, convert back to natural units
        if(convert_logs) {
            if(substr(mdl$varnm,1,5) == "log10") {
                ev <- 10^ev
                rl_cf <- 10^rl_cf
            } else if(substr(mdl$varnm,1,3) == "log") {
                ev <- exp(ev)
                rl_cf <- exp(rl_cf)
            }
        }
        c(pars, 
          "dispersion" = unname(pars["sigma0"] / pars["mu0"]),
          "variance" = unname(pars["sigma0"]),
          "event_magnitude" = ev,
          "return_period" = rp,
          "probability_ratio" = rp_cf/rp,
          "abs_change_in_intensity" = (ev - rl_cf),
          "rel_change_in_intensity" = ((ev - rl_cf) / rl_cf) * 100
         )   
    }
    
    # get best estimate from the observed data
    event_value <- mdl$x[mdl$ev_idx]
    mdl_res <- get_mdl_values(mdl, event_value, cov1, cov2, cov1_cf, cov2_cf)
    mdl_df <- setNames(data.frame(mdl$x, mdl$cov1, mdl$cov2), c(mdl$varnm, mdl$covnm_1, mdl$covnm_2)) 
    
    # get bootstrap sample
    set.seed(seed)    
    boot_res <- sapply(1:nsamp, function(i) {
        boot_df <- mdl_df[sample(1:nrow(mdl_df), nrow(mdl_df), replace = T),]
        boot_mdl <- fit_ns(mdl$dist, mdl$type, boot_df, varnm = mdl$varnm, covnm_1 = mdl$covnm_1, covnm_2 = mdl$covnm_2, lower = mdl$lower)
        get_mdl_values(boot_mdl, event_value, cov1, cov2, cov1_cf, cov2_cf)
    })
    boot_qq <- t(rbind("bestimate" = mdl_res, apply(boot_res, 1, quantile, c(0.025, 0.975), na.rm = T)))
    if(!is.na(dp)) boot_qq <- round(boot_qq, dp)
    
    if(spreadsheet_format) {
        return(c(setNames(boot_qq["dispersion",], c("disp_est", "disp_lower", "disp_upper")),
                 setNames(boot_qq["variance",], c("sigma_est", "sigma_lower", "sigma_upper")),
                 "event_magnitude" = boot_qq["event_magnitude", "bestimate"],
                 setNames(boot_qq["return_period",], c("rp_est", "rp_lower", "rp_upper")),
                 setNames(boot_qq["probability_ratio",], c("pr_est", "pr_lower", "pr_upper")),
                 setNames(boot_qq["rel_change_in_intensity",], c("rel_DI_est", "rel_DI_lower", "rel_DI_upper")),
                 setNames(boot_qq["abs_change_in_intensity",], c("abs_DI_est", "abs_DI_lower", "abs_DI_upper"))))
    } else {
        return(boot_qq)
    }
}

In [ ]:
pr_ond_res <- do.call("rbind", sapply(c("cpc", "chirps05+centrends01"), function(dsnm) {
    
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_pr-ond_",dsnm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    bootstrap_obs_results(fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T), nsamp = 1000,
                  cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, convert_logs = T)
}, simplify = F))
write.csv(pr_ond_res, "res/EA-drought_pr-ond_obs_fitted.csv")

In [ ]:
pr_ond_res_gmstonly <- do.call("rbind", sapply(c("cpc", "chirps05+centrends01"), function(dsnm) {
    
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_pr-ond_",dsnm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    bootstrap_obs_results(fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T), nsamp = 1000,
                  cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, convert_logs = T)
}, simplify = F))
write.csv(pr_ond_res_gmstonly, "res/EA-drought_pr-ond_obs_fitted-gmst-only.csv")

In [ ]:
pr_mam_res <- do.call("rbind", sapply(c("cpc", "chirps05+centrends01"), function(dsnm) {
    
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_pr-mam_",dsnm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    bootstrap_obs_results(fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T), nsamp = 1000,
                  cov1 = gmst_2022, cov1_cf = gmst_cf, convert_logs = T)
}, simplify = F))
write.csv(pr_mam_res, "res/EA-drought_pr-mam_obs_fitted.csv")

In [ ]:
pr_24_res <- do.call("rbind", sapply(c("cpc", "chirps05+centrends01"), function(dsnm) {
    
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_pr-24_",dsnm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    bootstrap_obs_results(fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T), nsamp = 1000,
                  cov1 = gmst_2022, cov1_cf = gmst_cf, convert_logs = T)
}, simplify = F))
write.csv(pr_24_res, "res/EA-drought_pr-24_obs_fitted.csv")

In [3]:
pet_24_res <- do.call("rbind", sapply(c("cpc"), function(dsnm) {
    
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_pet-24_",dsnm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pet")))
    
    bootstrap_obs_results(fit_ns("norm", "shift", pr_df, varnm = "pet", covnm_1 = "gmst", lower = F), nsamp = 1000,
                  cov1 = gmst_2022, cov1_cf = gmst_cf)
}, simplify = F))
write.csv(pet_24_res, "res/EA-drought_pet-24_obs_fitted.csv")

In [4]:
spei_24_res <- do.call("rbind", sapply(c("cpc"), function(dsnm) {
    
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_spei-24_",dsnm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "spei")))
    
    bootstrap_obs_results(fit_ns("norm", "shift", pr_df, varnm = "spei", covnm_1 = "gmst", lower = T), nsamp = 1000,
                  cov1 = gmst_2022, cov1_cf = gmst_cf)
}, simplify = F))
write.csv(spei_24_res, "res/EA-drought_spei-24_obs_fitted.csv")

#### **Values for contour plot**

In [ ]:
pr_df <- merge(merge(df, read.csv(paste0("ts/EA-drought_pr-24_cpc.dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr"))),
               read.csv(paste0("ts/EA-drought_pet-24_cpc.dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pet")))
pr_df["log10_pr"] <- log10(pr_df$pr)

mdl_pet <- fit_ns("norm", "shift", pr_df, varnm = "pet", covnm_1 = "gmst", lower = F)
mdl_pr <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T)

pet_cf <- map_from_u(map_to_u(mdl_pet)[mdl_pet$ev_idx], mdl_pet, cov1 = pr_df$gmst[pr_df$year == 2022] - 1.2)
pr_cf <- map_from_u(map_to_u(mdl_pr)[mdl_pr$ev_idx], mdl_pr, cov1 = pr_df$gmst[pr_df$year == 2022] - 1.2)
print(c(10^pr_cf, pet_cf))

### **Joint changes**

In [ ]:
# load data
pr_df <- merge(merge(merge(read.csv("ts/EA-drought_gmst-smoothed.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "gmst")),
                     read.csv(paste0("ts/EA-drought_pr-24_cpc.dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr"))),
               read.csv(paste0("ts/EA-drought_pet-24_cpc.dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pet"))),
               read.csv(paste0("ts/EA-drought_spei-24_cpc.dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "spei")))
pr_df["log10_pr"] <- log10(pr_df$pr)

gmst_2022 <- pr_df$gmst[pr_df$year == 2022]
gmst_cf <- gmst_2022 - 1.2

In [ ]:
# fit base models
mdl_pr <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T)
mdl_pet <- fit_ns("norm", "shift", pr_df, varnm = "pet", covnm_1 = "gmst", lower = F)
mdl_spei <- fit_ns("norm", "shift", pr_df, varnm = "spei", covnm_1 = "gmst", lower = T)

rp_pr <- 1/map_to_u(mdl_pr)[mdl_pr$ev_idx]
rp_pet <- 1/map_to_u(mdl_pet)[mdl_pet$ev_idx]
rp_spei <- 1/map_to_u(mdl_spei)[mdl_spei$ev_idx]

change_in_intensity <- c("pr" = Delta_I(mdl_pr, rp = rp_pr, cov1 = gmst_2022, cov1_cf = gmst_cf, relative = F),
                         "pet" = Delta_I(mdl_pet, rp = rp_pet, cov1 = gmst_2022, cov1_cf = gmst_cf, relative = F),
                         "spei" = Delta_I(mdl_spei, rp = rp_spei, cov1 = gmst_2022, cov1_cf = gmst_cf, relative = F))

In [ ]:
# bootstrap sample for change in intensity
nsamp <- 1000
set.seed(1)
boot_res <- sapply(1:nsamp, function(i) {
    
    # resample data, refit models
    boot_df <- pr_df[sample(1:nrow(pr_df), nrow(pr_df), replace = T),]
    bootmdl_pr <- fit_ns(mdl_pr$dist, mdl_pr$type, boot_df, varnm = mdl_pr$varnm, covnm_1 = mdl_pr$covnm_1, covnm_2 = mdl_pr$covnm_2, lower = mdl_pr$lower)
    bootmdl_pet <- fit_ns(mdl_pet$dist, mdl_pet$type, boot_df, varnm = mdl_pet$varnm, covnm_1 = mdl_pet$covnm_1, covnm_2 = mdl_pet$covnm_2, lower = mdl_pet$lower)
    bootmdl_spei <- fit_ns(mdl_spei$dist, mdl_spei$type, boot_df, varnm = mdl_spei$varnm, covnm_1 = mdl_spei$covnm_1, covnm_2 = mdl_spei$covnm_2, lower = mdl_spei$lower)
    
    c("pr" = Delta_I(bootmdl_pr, rp = rp_pr, cov1 = gmst_2022, cov1_cf = gmst_cf, relative = F),
      "pet" = Delta_I(bootmdl_pet, rp = rp_pet, cov1 = gmst_2022, cov1_cf = gmst_cf, relative = F),
      "spei" = Delta_I(bootmdl_spei, rp = rp_spei, cov1 = gmst_2022, cov1_cf = gmst_cf, relative = F))
})

In [ ]:
boxplot(t(boot_res))

In [ ]:
par(mfrow = c(1,3))
boxplot(t(boot_res[c("pr", "pet"),]))
boxplot(boot_res["pet",])

### **Plots of trend & return level**

In [ ]:
source("../wwa_nonstationary_fitting.r")

# load covariate data
df <- merge(read.csv("ts/EA-drought_gmst-smoothed.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "gmst")),
            read.csv("ts/EA-drought_detrended-nino-ond.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "nino_ond")))

# fix covariate values
gmst_2022 <- df[df$year == 2022, "gmst"]
gmst_cf <- gmst_2022 - 1.2

ondnino_2022 <- ondnino_cf <- df[df$year == 2022, "nino_ond"]

#### **pr_ond ~ gmst + nino_ond**

In [ ]:
varnm = "pr-ond"
sapply(c("cpc", "chirps05+centrends01"), function(ds_nm) {
        
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_",varnm,"_",ds_nm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    mdl <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T)

    png(paste0("fig/rlplot_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_returnperiods(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, ylab = "log10 precip")
        
        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
    
    png(paste0("fig/gmsttrend_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_gmsttrend(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, ylab = "log10 precip", ylim = c(1.6,2.8))

        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
})

In [ ]:
# or both models on one axis?
sapply(c("cpc", "chirps05+centrends01"), function(ds_nm) {
    
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_",varnm,"_",ds_nm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    mdl_nino <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T)
    mdl_gmst <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T)
    
    png(paste0("fig/gmsttrend_",varnm,"_",ds_nm,"_both-models.png"), width = 480*1.4, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_gmsttrend(mdl_gmst, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, legend_pos = NA)
        plot_gmsttrend(mdl_nino, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, ylab = "log10 precip", ylim = c(1.6,2.8), legend_pos = NA, add = T, col_main = "darkred", col_rl = "red", lty = "dotted")

        legend("topleft", legend = c("pr_ond ~ gmst", "1-in-6-year event", "1-in-40-year event", "pr_ond ~ gmst + nino", "1-in-6-year event", "1-in-40-year event"),
               ncol = 2, col = c("black", rep("blue", 2), "darkred", rep("red", 2)), lty = rep(1:2, each = 3), lwd = rep(c(2,2,1), 2), bty = "n")
        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
    
    png(paste0("fig/rlplot_",varnm,"_",ds_nm,"_both-models.png"), width = 480*1.4, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_returnperiods(mdl_gmst, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, ylab = "log10 precip", xlim = c(1,1000), ylim = c(1.6, 2.6), pch = NA, legend_pos = NA)
        plot_returnperiods(mdl_nino, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, add = T, legend_pos = NA, pch = NA, lty = 2)
        
        legend("topright", legend = c("pr_ond ~ gmst (2022 climate)", "pr_ond ~ gmst(counterfactual)", "pr_ond ~ gmst + nino (2022 climate)", "pr_ond ~ gmst + nino (counterfactual)"),
               ncol = 1, col = rep(c("firebrick", "blue"), 2), lty = rep(1:2, each = 2), bty = "n")
        
        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
})

#### **pr_ond ~ gmst**

In [ ]:
varnm = "pr-ond"
sapply(c("cpc", "chirps05+centrends01"), function(ds_nm) {
        
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_",varnm,"_",ds_nm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    mdl <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst",lower = T)

    png(paste0("fig/rlplot_",varnm,"-gmst_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_returnperiods(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, ylab = "log10 precip")
        
        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
    
    png(paste0("fig/gmsttrend_",varnm,"-gmst_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_gmsttrend(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, ylab = "log10 precip", ylim = c(1.6,2.8))

        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
})

#### **pr_mam ~ gmst**

In [ ]:
varnm = "pr-mam"
sapply(c("cpc", "chirps05+centrends01"), function(ds_nm) {
        
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_",varnm,"_",ds_nm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    mdl <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T)

    png(paste0("fig/rlplot_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_returnperiods(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, ylab = "log10 precip")
        
        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
    
    png(paste0("fig/gmsttrend_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_gmsttrend(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, ylab = "log10 precip", ylim = c(1.8,2.9), legend_pos = "topleft")

        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
})

#### **pr_24 ~ gmst**

In [ ]:
varnm = "pr-24"
sapply(c("cpc", "chirps05+centrends01"), function(ds_nm) {
        
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_",varnm,"_",ds_nm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pr")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    mdl <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T)

    png(paste0("fig/rlplot_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_returnperiods(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, ylab = "log10 precip")
        
        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
    
    png(paste0("fig/gmsttrend_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_gmsttrend(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, ylab = "log10 precip", ylim = c(2.7,3.2), legend_pos = NA)

        yticks <- c(50,seq(100,700,100))
        axis(4, at = log10(yticks), labels = yticks)
        mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    }; dev.off()
})

### **PET & SPEI**

In [ ]:
varnm = "pet-24"
sapply(c("cpc"), function(ds_nm) {
        
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_",varnm,"_",ds_nm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "pet")))
    
    mdl <- fit_ns("norm", "shift", pr_df, varnm = "pet", covnm_1 = "gmst", lower = F)

    png(paste0("fig/rlplot_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_returnperiods(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, ylab = "PET (mm)", ylim = c(80,150))
    }; dev.off()
    
    png(paste0("fig/gmsttrend_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_gmsttrend(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, ylab = "PET (mm)", legend_pos = NA, ylim = c(80,150))
    }; dev.off()
})

In [ ]:
varnm = "spei-24"
sapply(c("cpc"), function(ds_nm) {
        
    pr_df <- merge(df, read.csv(paste0("ts/EA-drought_",varnm,"_",ds_nm,".dat"), comment.char = "#", sep = " ", header = F, col.names = c("year", "spei")))
    
    mdl <- fit_ns("norm", "shift", pr_df, varnm = "spei", covnm_1 = "gmst", lower = T)

    png(paste0("fig/rlplot_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_returnperiods(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, ylab = "24-month SPEI", ylim = c(-3.9, 3.9))
    }; dev.off()
    
    png(paste0("fig/gmsttrend_",varnm,"_",ds_nm,".png"), width = 480*1.2, height = 480); {
        par(mar = c(3.5,3.5,1,3.5), cex = 1.4)
        plot_gmsttrend(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, ylab = "24-month SPEI", legend_pos = NA, ylim = c(-3.9, 3.9))
    }; dev.off()
})

## **Station data**

In [ ]:
source("../wwa_nonstationary_fitting.r")

# load covariate data
df <- merge(read.csv("ts/EA-drought_gmst-smoothed.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "gmst")),
            read.csv("ts/EA-drought_detrended-nino-ond.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "nino_ond")))

# fix covariate values
gmst_2022 <- df[df$year == 2022, "gmst"]
gmst_cf <- gmst_2022 - 1.2

ondnino_2022 <- ondnino_cf <- df[df$year == 2022, "nino_ond"]

### **Tables of fitted trends**

In [ ]:
stn_ond_res <- t(sapply(list.files("ts", pattern = "ond_stn", full.names = T), function(fnm) {
    
    stn <- gsub(".dat", "", strsplit(fnm, "-")[[1]][5])
    pr_df <- merge(df, read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "precip")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    bootstrap_obs_results(fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T), nsamp = 1000,
                          cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, convert_logs = T)
}))
write.csv(stn_ond_res, "res/EA-drought_pr-ond_stations_fitted.csv")

In [ ]:
stn_mam_res <- t(sapply(list.files("ts", pattern = "mam_stn", full.names = T), function(fnm) {
    
    stn <- gsub(".dat", "", strsplit(fnm, "-")[[1]][5])
    pr_df <- merge(df, read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "precip")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    bootstrap_obs_results(fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T), nsamp = 1000,
                          cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, convert_logs = T)
}))
write.csv(stn_mam_res, "res/EA-drought_pr-mam_stations_fitted.csv")

In [ ]:
stn_24_res <- t(sapply(list.files("ts", pattern = "24-month_stn", full.names = T), function(fnm) {
    
    stn <- gsub(".dat", "", strsplit(fnm, "-")[[1]][5])
    pr_df <- merge(df, read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "precip")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    bootstrap_obs_results(fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T), nsamp = 1000,
                          cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, convert_logs = T)
}))
write.csv(stn_mam_res, "res/EA-drought_pr-24_stations_fitted.csv")

### **Trend plots**

In [ ]:
# Trends in GMST
prep_window(c(2,4))
par(mar = c(3.5,3.5,3.5,3.5))
invisible(sapply(list.files("ts", pattern = "24-month_stn", full.names = T), function(fnm) {
    
    stn <- gsub(".dat", "", strsplit(fnm, "-")[[1]][5])
    pr_df <- merge(df, read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "precip")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    mdl <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T)
    
    plot_gmsttrend(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, ylab = "log10 precip", legend_pos = NA, main = stn)
    yticks <- c(50,seq(100,1500,100), seq(2000,5000,500))
    axis(4, at = log10(yticks), labels = yticks)
    mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
}))
plot.new()
legend("left", legend = c("location", "1-in-6-year event", "1-in-40-year event"), lty = 1, col = c("black", rep("blue", 2)), lwd = c(2,2,1))

In [ ]:
# Return level plots
prep_window(c(2,4))
par(mar = c(3.5,3.5,3.5,3.5))
invisible(sapply(list.files("ts", pattern = "24-month_stn", full.names = T), function(fnm) {
    
    stn <- gsub(".dat", "", strsplit(fnm, "-")[[1]][5])
    pr_df <- merge(df, read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "precip")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    mdl <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T)
    
    plot_returnperiods(mdl, cov1 = gmst_2022, cov1_cf = gmst_cf, cov2 = ondnino_2022, cov2_cf = ondnino_cf, ylab = "log10 precip", main = stn)
    
    yticks <- c(50,seq(100,1500,100), seq(2000,5000,500))
    axis(4, at = log10(yticks), labels = yticks)
    mtext("Precip (mm)", side = 4, line = 2.5, cex = par("cex"))
    
}))

In [ ]:
# Trends over time

prep_window(c(2,4))
par(mar = c(3.5,3.5,3.5,3.5))
invisible(sapply(list.files("ts", pattern = "24-month_stn", full.names = T), function(fnm) {
    
    stn <- gsub(".dat", "", strsplit(fnm, "-")[[1]][5])
    pr_df <- merge(df, read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "precip")))
    pr_df["log10_pr"] <- log10(pr_df$pr)
    
    mdl <- fit_ns("norm", "fixeddisp", pr_df, varnm = "log10_pr", covnm_1 = "gmst", lower = T)
    rp_2022 <- 1/map_to_u(mdl)[mdl$ev_idx]
    
    plot(pr_df[,c("year", "precip")], type = "S", lwd = 2, xlab = "", ylab = "", col = adjustcolor("black", alpha = 0.5), main = paste0(stn, " (rp", round(rp_2022,0),")"))
    lines(pr_df$year, 10^ns_pars(mdl)$loc, type = "S", lwd = 2, col = "black")
    lines(pr_df$year, 10^map_from_u(1/6, mdl, cov1 = mdl$cov1), col = "blue", lwd = 2, type = "s")
    lines(pr_df$year, 10^map_from_u(1/40, mdl, cov1 = mdl$cov1), col = "blue", lwd = 1, type = "s")
    
    points(2022, pr_df$precip[pr_df$year == 2022], col = "magenta", pch = 20, cex = 2)
    mtext("Precip (mm)", side = 2, line = 2.5, cex = par("cex"))
}))
plot.new()
legend("left", legend = c("location", "1-in-6-year event", "1-in-40-year event"), lty = 1, col = c("black", rep("blue", 2)), lwd = c(2,2,1))

---
## **Models**

In [1]:
source("../wwa_nonstationary_fitting.r")

model_results <- function(df, dist, fittype, varnm, covnm_1, covnm_2 = NA, cov1_2022, cov1_hist, cov1_fut, cov2_2022 = 0, cov2_hist = 0, cov2_fut = 0,
                                lower = F, rp = 10, delta_relative = T, nsamp = 1000, seed = 1) {
    
    # FUNCTION TO CARRY OUT ALL ATTRIBUTION RUNS & PRODUCE RESULTS IN SPREADSHEET-FRIENDLY FORM
    
    set.seed(seed)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FIT MODEL to three different subsets: for evaluation, attribution & projection
    
    df_eval <- df[df$year >= 1979 & df$year <= 2022,]
    df_attr <- df[df$year <= 2022,]
    df_proj <- df[df$year <= 2050,]
    
    mdl_eval <- fit_ns(dist, fittype, df_eval, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    mdl_attr <- fit_ns(dist, fittype, df_attr, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    mdl_proj <- fit_ns(dist, fittype, df_proj, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## EVALUATION (get model parameters)
    
    if(fittype == "fixeddisp") {
        disp_eval <- mdl_eval$par["sigma0"] / mdl_eval$par["mu0"]
        boot_eval <- sapply(1:nsamp, function(i) {
            
            boot_df <- df_eval[sample(1:nrow(df_eval), nrow(df_eval), replace = T),]
            boot_mdl <- fit_ns(mdl_eval$dist, mdl_eval$type, boot_df, varnm = mdl_eval$varnm, covnm_1 = mdl_eval$covnm_1, covnm_2 = mdl_eval$covnm_2, lower = mdl_eval$lower)
            boot_mdl$par["sigma0"] / boot_mdl$par["mu0"]
        })
        ci_eval <- setNames(c(disp_eval, quantile(boot_eval, c(0.025, 0.975))),
                            c("disp_est", "disp_lower", "disp_upper"))
    } else {
        var_eval <- mdl_eval$par["sigma0"]
        boot_eval <- sapply(1:nsamp, function(i) {
            
            boot_df <- df_eval[sample(1:nrow(df_eval), nrow(df_eval), replace = T),]
            boot_mdl <- fit_ns(mdl_eval$dist, mdl_eval$type, boot_df, varnm = mdl_eval$varnm, covnm_1 = mdl_eval$covnm_1, covnm_2 = mdl_eval$covnm_2, lower = mdl_eval$lower)
            boot_mdl$par["sigma0"]
        })
        ci_eval <- setNames(c(var_eval, quantile(boot_eval, c(0.025, 0.975))),
                            c("var_est", "var_lower", "var_upper"))
    }
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## ATTRIBUTION (get event value, PR & DI)
    
    rl_2022 <- map_from_u(1/rp, mdl_attr, cov1 = cov1_2022, cov2 = cov2_2022)
    if(substr(varnm,1,5) == "log10") { actual_rl2022 <- 10^rl_2022 } else { actual_rl2022 <- rl_2022 }
    
    pr_eval <- prob_ratio(mdl_attr, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist)
    dI_eval <- Delta_I(mdl_attr, rp, cov1 = cov1_2022, cov1_cf = cov1_hist, relative = delta_relative)
    
    boot_attr <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_attr[sample(1:nrow(df_attr), nrow(df_attr), replace = T),]
        boot_mdl <- fit_ns(mdl_attr$dist, mdl_attr$type, boot_df, varnm = mdl_attr$varnm, covnm_1 = mdl_attr$covnm_1, covnm_2 = mdl_attr$covnm_2, lower = mdl_attr$lower)
        
        boot_pr <- prob_ratio(boot_mdl, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist)
        boot_di <- Delta_I(boot_mdl, 10, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist, relative = delta_relative)
        
        c("PR" = boot_pr, "DI" = boot_di)
    })
    ci_attr <- rbind("est" = c(pr_eval, dI_eval), apply(boot_attr, 1, quantile, c(0.025, 0.975)))
    ci_attr <- unlist(lapply(colnames(ci_attr), function(cnm) setNames(ci_attr[,cnm], paste("attr", cnm, c("est", "lower", "upper"), sep = "_"))))
                          
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## PROJECTION (get PR & DI)

    pr_proj <- prob_ratio(mdl_proj, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_fut, cov2 = cov2_2022, cov2_cf = cov2_fut)
    dI_proj <- Delta_I(mdl_proj, 10, cov1 = cov1_2022, cov1_cf = cov1_fut, relative = delta_relative)
                             
    boot_proj <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_proj[sample(1:nrow(df_proj), nrow(df_proj), replace = T),]
        boot_mdl <- fit_ns(mdl_proj$dist, mdl_proj$type, boot_df, varnm = mdl_proj$varnm, covnm_1 = mdl_proj$covnm_1, covnm_2 = mdl_proj$covnm_2, lower = mdl_proj$lower)
        
        boot_pr <- prob_ratio(boot_mdl, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_fut, cov2 = cov2_2022, cov2_cf = cov2_fut)
        boot_di <- Delta_I(boot_mdl, 10, cov1 = cov1_2022, cov1_cf = cov1_fut, cov2 = cov2_2022, cov2_cf = cov2_fut, relative = delta_relative)
        
        c("PR" = boot_pr, "DI" = boot_di)
    })
    ci_proj <- rbind("est" = c(pr_proj, dI_proj), apply(boot_proj, 1, quantile, c(0.025, 0.975)))
      
    # invert
    ci_proj[,"PR"] <- 1/ci_proj[c(1,3,2),"PR"]
    ci_proj[,"DI"] <- -ci_proj[c(1,3,2),"DI"]   
                             
    ci_proj <- unlist(lapply(colnames(ci_proj), function(cnm) setNames(ci_proj[,cnm], paste("proj", cnm, c("est", "lower", "upper"), sep = "_"))))
                             
    # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                             
    # t(setNames(data.frame(c(ci_eval, "rp_value" = actual_rl2022, ci_attr, ci_proj)), gsub(".dat", "", gsub(".+pr-mam_", "", fnm))))      
    t(data.frame(c(ci_eval, "rp_value" = actual_rl2022, ci_attr, ci_proj)))     
}



### **CORDEX**

#### **Precipitation indices**

In [ ]:
# fit model & get results for each season
invisible(sapply(c("mam", "ond", "24"), function(seas) {
    
    rp <- list("mam" = 10, "ond" = 5, "24" = 20)[[seas]]                  # return period depending on variable
    if(seas == "ond") { covnm_2 = "nino_ond" } else { covnm_2 = NA }      # OND needs second covariate
    
    # loop over all available models, get results for each
    res <- sapply(list.files("ts", pattern = paste0("pr-",seas,"_AFR"), full.names = T), function(fnm) {
        
        if(grepl("HadGEM2-ES_r1_RegCM4-3", fnm)) {return(rep(NA, 16))}
        
        # load necessary data
        gmst_fnm <- list.files("ts", pattern = paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4],"_rcp85_",strsplit(fnm, "_")[[1]][5],".dat"), full.names = T)
        gmst <- read.csv(gmst_fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "gmst"))
        
        nino_fnm <- list.files("ts", paste0("nino-ond_",strsplit(fnm, "_")[[1]][4],"_rcp85_",strsplit(fnm, "_")[[1]][5],".dat"), full.names = T)
        nino <- read.csv(nino_fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "nino_ond"))
        
        pr <- read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "pr"))
        df <- merge(merge(gmst, nino), pr)
        df["log10pr"] <- log10(df$pr)   # fit models to log precip, convert back at the end (more stable model fitting)
        
        # covariates defining current, past & future climates
        gmst_2022 <- gmst[gmst$year == 2022, "gmst"]
        gmst_hist <- gmst_2022 - 1.2
        gmst_fut <- gmst_2022 + 0.8
        nino_2022 <- nino_hist <- nino_fut <- 0    # dummy covariates
        
        # fit model & get results
        data.frame(model_results(df, "norm", "fixeddisp", varnm = "log10pr", covnm_1 = "gmst", covnm_2 = covnm_2, lower = T, rp = rp, delta_relative = T, nsamp = 1000,
                                       cov1_2022 = gmst_2022, cov1_hist = gmst_hist, cov1_fut = gmst_fut, 
                                       cov2_2022 = nino_2022, cov2_hist = nino_hist, cov2_fut = nino_fut))
    }, simplify = F)
    
    # merge results into single dataframe, tidy rownames & save to csv
    res <- do.call("rbind", res)
    rownames(res) <- gsub(".dat", "", gsub(paste0(".+pr-",seas,"_"), "", rownames(res)))
    write.csv(res, paste0("res/EA-drought_pr-",seas,"_cordex_fitted.csv"))
}))

#### **PET & SPEI**

In [3]:
# fit model & get results for each season
invisible(sapply(c("pet", "spei")[1], function(varnm) {
    
    rp <- list("pet" = 2, "spei" = 10)[[varnm]]                  # return period depending on variable
    lower <- list("pet" = F, "spei" = T)[[varnm]]                # consider lower or upper tail, depending on variable
    relative_delta <- list("pet" = T, "spei" = F)[[varnm]]
    
    # loop over all available models, get results for each
    res <- sapply(sort(list.files("ts", pattern = paste0("_",varnm,"-24_AFR"), full.names = T)), function(fnm) {
        
        if(grepl("HadGEM2-ES_r1_RegCM4-3", fnm)) {return(rep(NA, 16))}
        
        # load necessary data
        gmst_fnm <- list.files("ts", pattern = paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4],"_rcp85_",strsplit(fnm, "_")[[1]][5],".dat"), full.names = T)
        gmst <- read.csv(gmst_fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "gmst"))
        
        pr <- read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", varnm))
        df <- merge(gmst, pr)
        df <- df[!is.na(df[,varnm]),]
        
        # covariates defining current, past & future climates
        gmst_2022 <- gmst[gmst$year == 2022, "gmst"]
        gmst_hist <- gmst_2022 - 1.2
        gmst_fut <- gmst_2022 + 0.8
        nino_2022 <- nino_hist <- nino_fut <- 0    # dummy covariates
        
        # fit model & get results
        data.frame(model_results(df, "norm", "shift", varnm = varnm, covnm_1 = "gmst", covnm_2 = NA, lower = lower, rp = rp, delta_relative = relative_delta, nsamp = 1000,
                                       cov1_2022 = gmst_2022, cov1_hist = gmst_hist, cov1_fut = gmst_fut, 
                                       cov2_2022 = nino_2022, cov2_hist = nino_hist, cov2_fut = nino_fut))
    }, simplify = F)
    
    # merge results into single dataframe, tidy rownames & save to csv
    res <- do.call("rbind", res)
    rownames(res) <- gsub(".dat", "", gsub(".+-24_", "", rownames(res)))
    write.csv(res, paste0("res/EA-drought_",varnm,"-24_cordex_fitted.csv"))
}))

### **HighResMIP**

In [ ]:
source("../wwa_nonstationary_fitting.r")

# load covariate data
cov_df <- merge(read.csv("ts/EA-drought_gmst-smoothed.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "gmst")),
            read.csv("ts/EA-drought_detrended-nino-ond.dat", comment.char = "#", sep = " ", header = F, col.names = c("year", "nino_ond")))

# fix covariate values defining current, past & future climates
gmst_2022 <- cov_df[cov_df$year == 2022, "gmst"]
gmst_hist <- gmst_2022 - 1.2
gmst_fut <- gmst_2022 + 0.8

nino_2022 <- nino_hist <- nino_fut <- cov_df[cov_df$year == 2022, "nino_ond"]

In [ ]:
hiresmip_results <- function(df, dist, fittype, varnm, covnm_1, covnm_2 = NA, cov1_2022, cov1_hist, cov1_fut = NA, cov2_2022 = 0, cov2_hist = 0, cov2_fut = NA,
                                lower = F, rp = 10, delta_relative = T, nsamp = 1000, seed = 1) {
    
    # FUNCTION TO CARRY OUT ALL ATTRIBUTION RUNS & PRODUCE RESULTS IN SPREADSHEET-FRIENDLY FORM
    
    set.seed(seed)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FIT MODEL to three different subsets: for evaluation, attribution & projection
    
    df_eval <- df[df$year >= 1979 & df$year <= 2022,]
    df_attr <- df[df$year <= 2022,]
    
    mdl_eval <- fit_ns(dist, fittype, df_eval, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    mdl_attr <- fit_ns(dist, fittype, df_attr, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## EVALUATION (get model parameters)
    
    disp_eval <- mdl_eval$par["sigma0"] / mdl_eval$par["mu0"]
    var_eval <- mdl_eval$par["sigma0"]
    boot_eval <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_eval[sample(1:nrow(df_eval), nrow(df_eval), replace = T),]
        boot_mdl <- fit_ns(mdl_eval$dist, mdl_eval$type, boot_df, varnm = mdl_eval$varnm, covnm_1 = mdl_eval$covnm_1, covnm_2 = mdl_eval$covnm_2, lower = mdl_eval$lower)
        if(fittype == "fixeddisp") {
            boot_mdl$par["sigma0"] / boot_mdl$par["mu0"]
        } else {
            boot_mdl$par["sigma0"]
        }
    })
    if(fittype == "fixeddisp") {
        ci_eval <- setNames(c(disp_eval, quantile(boot_eval, c(0.025, 0.975))),
                            c("disp_est", "disp_lower", "disp_upper"))
    } else {
        ci_eval <- setNames(c(var_eval, quantile(boot_eval, c(0.025, 0.975))),
                            c("var_est", "var_lower", "var_upper"))
    }
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## ATTRIBUTION (get event value, PR & DI)
    
    rl_2022 <- map_from_u(1/rp, mdl_attr, cov1 = cov1_2022, cov2 = cov2_2022)
    if(substr(varnm,1,5) == "log10") { actual_rl2022 <- 10^rl_2022 } else { actual_rl2022 <- rl_2022 }
    
    pr_eval <- prob_ratio(mdl_attr, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist)
    dI_eval <- Delta_I(mdl_attr, rp, cov1 = cov1_2022, cov1_cf = cov1_hist, relative = delta_relative)
    
    boot_attr <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_attr[sample(1:nrow(df_attr), nrow(df_attr), replace = T),]
        boot_mdl <- fit_ns(mdl_attr$dist, mdl_attr$type, boot_df, varnm = mdl_attr$varnm, covnm_1 = mdl_attr$covnm_1, covnm_2 = mdl_attr$covnm_2, lower = mdl_attr$lower)
        
        boot_pr <- prob_ratio(boot_mdl, rl_2022, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist)
        boot_di <- Delta_I(boot_mdl, rp, cov1 = cov1_2022, cov1_cf = cov1_hist, cov2 = cov2_2022, cov2_cf = cov2_hist, relative = delta_relative)
        
        c("PR" = boot_pr, "DI" = boot_di)
    })
    ci_attr <- rbind("est" = c(pr_eval, dI_eval), apply(boot_attr, 1, quantile, c(0.025, 0.975)))
    ci_attr <- unlist(lapply(colnames(ci_attr), function(cnm) setNames(ci_attr[,cnm], paste("attr", cnm, c("est", "lower", "upper"), sep = "_"))))
                             
    # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Can't do future projection, because using observed GMST & Nino time series as covariates
                             
    # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                             
    # t(setNames(data.frame(c(ci_eval, "rp_value" = rl_2022, ci_attr, ci_proj)), gsub(".dat", "", gsub(".+pr-mam_", "", fnm))))      
    t(data.frame(c(ci_eval, "rp_value" = actual_rl2022, ci_attr)))     
}

#### **Precipitation indices**

In [ ]:
# fit model & get results for each season
invisible(sapply(c("mam", "ond", "24"), function(seas) {
    
    rp <- list("mam" = 10, "ond" = 5, "24" = 20)[[seas]]                  # return period depending on variable
    if(seas == "ond") { covnm_2 = "nino_ond" } else { covnm_2 = NA }      # OND needs second covariate
    
    # loop over all available models, get results for each
    res <- sapply(list.files("ts", pattern = paste0("pr-",seas,"_highres"), full.names = T), function(fnm) {
        
        pr <- read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "pr"))
        df <- merge(cov_df, pr)
        df["log10pr"] <- log10(df$pr)   # fit models to log precip, convert back at the end (more stable model fitting)
        
        data.frame(hiresmip_results(df, "norm", "fixeddisp", varnm = "log10pr", covnm_1 = "gmst", covnm_2 = covnm_2, lower = T, rp = rp, delta_relative = T, nsamp = 1000,
                                       cov1_2022 = gmst_2022, cov1_hist = gmst_hist, cov1_fut = gmst_fut, 
                                       cov2_2022 = nino_2022, cov2_hist = nino_hist, cov2_fut = nino_fut))
    }, simplify = F)
    
    # merge results into single dataframe, tidy rownames & save to csv
    res <- do.call("rbind", res)
    rownames(res) <- gsub(".dat", "", gsub(".+highresSST_", "", rownames(res)))
    write.csv(res, paste0("res/EA-drought_pr-",seas,"_highresmip_fitted.csv"))
}))

#### **PET & SPEI**

In [ ]:
invisible(sapply(c("pet", "spei"), function(varnm) {
    
    rp <- list("pet" = 2, "spei" = 15)[[varnm]]                  # return period depending on variable
    lower <- list("pet" = F, "spei" = T)[[varnm]]                # consider lower or upper tail, depending on variable
    relative_delta <- list("pet" = T, "spei" = F)[[varnm]]
    
    # loop over all available models, get results for each
    res <- sapply(list.files("ts", pattern = paste0(varnm, "-24_highres"), full.names = T), function(fnm) {
        
        ts <- read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", varnm))
        ts <- ts[!is.na(ts[,varnm]),]   # only fit to finite SPEI values (avoids problem of truncation of high values)
        df <- merge(cov_df, ts)
        
        data.frame(hiresmip_results(df, "norm", "shift", varnm = varnm, covnm_1 = "gmst", lower = lower, rp = rp, delta_relative = relative_delta, nsamp = 1000,
                                       cov1_2022 = gmst_2022, cov1_hist = gmst_hist, cov1_fut = gmst_2022, 
                                       cov2_2022 = nino_2022, cov2_hist = nino_hist, cov2_fut = gmst_2022))
        }, simplify = F)
    
    # merge results into single dataframe, tidy rownames & save to csv
    res <- do.call("rbind", res)
    rownames(res) <- gsub(".dat", "", gsub(".+highresSST_", "", rownames(res)))
    write.csv(res, paste0("res/EA-drought_",varnm,"-24_highresmip_fitted.csv"))
}))